# Datos entreneamiento Shift

En este notebook vamos a aplicar el shift para las obtener las columnas mas correlacionadas.

- __Datos recibidos:__ obtenidos de notebook __1-datos_entrenamiento_shift.ipynb__
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

__Notas del proyecto__
Necesitamos generar un csv con los niveles máximos, mínimo y promedio por día, se perderá la información de la estación, también generaremos el atributo PM10mean y PM25mean que representarán los valores de las últimas 24 horas de los contaminantes.

calidad de caire
pronostico contaminación

## Pasos
[X] Aplicar Shift con Correlaciones <br>
[X] Sacar minimo promedio y maximo <br>
[X] Obtener columnas más correlacionadas<br><br>

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None 

In [2]:
path_inicial = "/home/paw/DanielBustillos/contaminación/pipeline-norberto/pipeline_pau/datos/ingesta_hora_limpios.csv"
path_final = "/home/paw/DanielBustillos/contaminación/pipeline-norberto/pipeline_pau/datos/ingesta_modelos_O3.csv"

In [3]:
data_hour_merge_24 = pd.read_csv(path_inicial)

In [4]:
data_hour_merge_24.columns

Index(['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', 'PM2.5', 'RH', 'SO2', 'TMP',
       'WDR', 'WSP', 'dia', 'fecha', 'hora', 'id_station', 'mes', 'PM10mean',
       'PM25mean'],
      dtype='object')

Quedémonos sin WSR:

In [5]:
data_hour_merge_24 = data_hour_merge_24[['CO', 'NO', 'NO2', 'NOX', 'O3', 'PM10', "PM2.5",'RH', 'SO2', 'TMP',
       'WSP', 'dia', 'fecha', 'hora', 'id_station', 'mes',"PM10mean","PM25mean"]].reset_index(drop=True)

Hagamos una lista con todas las estaciones:

### Arreglemos las fechas:

In [6]:
data_hour_merge_24['dia'] = data_hour_merge_24['dia'].astype(str).astype(float)
data_hour_merge_24['mes'] = data_hour_merge_24['mes'].astype(str).astype(float)

Creemos un diccionario con __data_hour_merge_24__ dividido por estación:

In [7]:
estaciones = data_hour_merge_24.id_station.unique().tolist()

In [8]:
data_est = {}
for elem in estaciones:
    data_est[elem] = data_hour_merge_24[data_hour_merge_24.id_station == elem]

# Leamos el archivo de correlaciones del PM10 mean

Este archivo se generó en el notebook: __/home/paw/DanielBustillos/contaminación/correlaciones_pau/correlaciones_función_paulina.ipynb__

In [9]:
target = "O3_y"

In [10]:
O3_corr = pd.read_csv("/DATA/paw/jupyterhub_notebook/daniel.bustillos/DanielBustillos/contaminación/pipeline-norberto/correlacion/correlaciones_"+target+".csv")
O3_corr = O3_corr.iloc[:,1:]
O3_corr.head(5)

,contaminante,valor,horas
0,hora,0.330936,0.0
1,dia,-0.006618,0.0
2,mes,-0.185332,0.0
3,CO,-0.120668,0.0
4,NO,-0.317031,0.0


Vamos a quedarnos con las variables con corr>0.3:

In [11]:
O3_corr_filtro = O3_corr[(O3_corr.valor>0.4) | (O3_corr.valor<-0.4) ].reset_index(drop=True)

In [12]:
O3_corr_filtro.shape

(147, 3)

Vamos a aplicar el shift para cada elemento de la tabla __O3_corr__:

In [13]:
def shit_corr(df):
    for i in range(len(O3_corr_filtro)):
        name_column = str( O3_corr_filtro.loc[i,"contaminante"] + "_" + str( O3_corr_filtro.loc[i,"horas"] ) ) 
        df[name_column] = df[O3_corr_filtro.loc[i,"contaminante"]].shift( - int(float(str(O3_corr_filtro.loc[i,"horas"]))) ) 
    return df.dropna()

Aplicamos esta función para cada DF de cada estación:

In [14]:
data_shift = {}
for elem in data_est:
    data_shift[elem] = shit_corr(data_est[elem])

Creamos un Df vacío:

In [15]:
df_append = pd.DataFrame(columns = data_est["MER"].columns.tolist())

In [16]:
for key in data_est:
    df_append = df_append.append(data_est[key], ignore_index=True,sort=True)

## Groupbys

In [17]:
cols = df_append.columns.tolist()

In [18]:
cols.remove("id_station")

Finalmente, aplicamos los groupbys:

In [19]:
data_hour_merge_24_mean = df_append.groupby(['fecha',"hora","dia","mes"]).mean()
data_hour_merge_24_mean.reset_index(inplace=True)

In [20]:
data_hour_merge_24_max = df_append.groupby(['fecha',"hora","dia","mes"]).max()
data_hour_merge_24_max.reset_index(inplace=True)

In [21]:
data_hour_merge_24_min = df_append.groupby(['fecha',"hora","dia","mes"]).min()
data_hour_merge_24_min.reset_index(inplace=True)

### Arreglemos los nombres:

In [22]:
max_columns = data_hour_merge_24_max.columns.tolist()
min_columns = data_hour_merge_24_min.columns.tolist()
mean_columns = data_hour_merge_24_mean.columns.tolist()

In [23]:
for i in range(len(max_columns)):
    if max_columns[i] not in ['fecha','hora','dia','mes','id_station']:
        max_columns[i] = max_columns[i]+"_max"
        min_columns[i] = min_columns[i]+"_min"    
        mean_columns[i] = mean_columns[i]+"_mean"

In [24]:
data_hour_merge_24_mean.columns = mean_columns
data_hour_merge_24_min.columns = min_columns
data_hour_merge_24_max.columns = max_columns

##### Unamos los df's de manera que tengamos en una solo los datos promedio, máximo y minimo por día:

In [25]:
data_hour_merge = pd.merge(data_hour_merge_24_mean, data_hour_merge_24_max, on=['fecha',"hora","dia","mes"])
data_hour_merge = pd.merge(data_hour_merge, data_hour_merge_24_min, on=['fecha',"hora","dia","mes"])

In [26]:
data_hour_merge = data_hour_merge.sort_values(['fecha',"hora"], ascending=[0,1])

## Leamos el archivo:

## Correlacionados con el target.

Guardemos estas variables, no pueden cambiarse una vez que los modelos estan entrenados:

Filtremos el DF con las variables mas correlacionadas:

In [27]:
O3_corr = ['O3_25.0_mean',  'O3_22.0_mean',  'TMP_48.0_mean',  'RH_mean',  'O3_max',  'TMP_1.0_min',  
           'O3_0.0_max',  'O3_44.0_min',  'O3_45.0_max',  'O3_21.0_min',  'hora_10.0_mean',  
           'TMP_23.0_mean',  'RH_min',  'hora_13.0_mean',  'O3_0.0_mean',  'O3_44.0_max', 
           'RH_24.0_mean',  'hora_13.0_max',  'O3_20.0_mean',  'O3_0.0_min',  'RH_max', 
           'O3_24.0_mean',  'TMP_21.0_mean',  'O3_21.0_max',  'O3_mean',  'O3_22.0_min', 
           'RH_45.0_mean',  'TMP_max',  'O3_23.0_max',  'O3_2.0_mean',  'TMP_47.0_max', 
           'O3_47.0_max',  'O3_2.0_max',  'TMP_24.0_mean',  'TMP_23.0_max',  'RH_22.0_min', 
           'O3_3.0_max',  'O3_21.0_mean',  'O3_47.0_mean',  'hora_11.0_mean',  'TMP_0.0_max', 
           'RH_2.0_min',  'RH_21.0_mean',  'RH_21.0_min',  'hora_11.0_max',  'hora_12.0_min', 
           'O3_48.0_mean',  'RH_22.0_max',  'RH_25.0_mean',  'O3_1.0_min',  'O3_46.0_max', 
           'RH_0.0_mean',  'O3_25.0_max',  'O3_2.0_min',  'RH_45.0_min',  'RH_46.0_mean', 
           'TMP_25.0_mean',  'TMP_0.0_min',  'O3_1.0_max',  'RH_23.0_mean',  'hora_36.0_max', 
           'RH_22.0_mean',  'O3_24.0_max',  'TMP_0.0_mean',  'RH_1.0_max',  'TMP_22.0_mean',  
           'WSP_mean',  'TMP_22.0_max',  'TMP_25.0_max',  'TMP_1.0_mean',  'O3_46.0_min', 
           'hora_12.0_max',  'hora_36.0_mean',  'O3_45.0_min',  'O3_46.0_mean',  'O3_1.0_mean', 
           'hora_34.0_mean',  'TMP_2.0_max',  'TMP_mean',  'TMP_1.0_max',  'TMP_3.0_max',  
           'TMP_47.0_mean',  'RH_0.0_max',  'RH_46.0_min',  'TMP_44.0_mean',  'O3_min', 
           'TMP_45.0_mean',  'O3_22.0_max',  'TMP_24.0_max',  'O3_23.0_min',  'RH_0.0_min', 
           'RH_1.0_min',  'TMP_46.0_mean',  'O3_44.0_mean',  'hora_11.0_min',  'RH_23.0_min', 
           'RH_2.0_mean',  'TMP_21.0_max',  'RH_1.0_mean',  'TMP_46.0_max',  'RH_24.0_min',  
           'O3_45.0_mean',  'TMP_2.0_mean',  'RH_47.0_mean',  'O3_24.0_min',  'hora_35.0_mean',  
           'hora_12.0_mean',  'O3_20.0_min',  'O3_20.0_max',  'O3_23.0_mean',  'TMP_45.0_max', 
           'TMP_min',  'RH_44.0_mean',  'RH_mean_frcst_1',  'RH_mean_frcst_2',  'RH_mean_frcst_3', 
           'RH_mean_frcst_4',  'RH_mean_frcst_5',  'RH_mean_frcst_6',  'RH_mean_frcst_7',  
           'RH_mean_frcst_8',  'RH_mean_frcst_9',  'RH_mean_frcst_10',  'RH_mean_frcst_11', 
           'RH_mean_frcst_12',  'RH_mean_frcst_13',  'RH_mean_frcst_14',  'RH_mean_frcst_15',  
           'RH_mean_frcst_16',  'RH_mean_frcst_17',  'RH_mean_frcst_18',  'RH_mean_frcst_19', 
           'RH_mean_frcst_20',  'RH_mean_frcst_21',  'RH_mean_frcst_22',  'RH_mean_frcst_23',  
           'RH_mean_frcst_24',  'WSP_mean_frcst_1',  'WSP_mean_frcst_2',  'WSP_mean_frcst_3', 
           'WSP_mean_frcst_4',  'WSP_mean_frcst_5',  'WSP_mean_frcst_6',  'WSP_mean_frcst_7',  
           'WSP_mean_frcst_8',  'WSP_mean_frcst_9',  'WSP_mean_frcst_10',  'WSP_mean_frcst_11',
           'WSP_mean_frcst_12',  'WSP_mean_frcst_13',  'WSP_mean_frcst_14',  'WSP_mean_frcst_15', 
           'WSP_mean_frcst_16',  'WSP_mean_frcst_17',  'WSP_mean_frcst_18',  'WSP_mean_frcst_19', 
           'WSP_mean_frcst_20',  'WSP_mean_frcst_21',  'WSP_mean_frcst_22',  'WSP_mean_frcst_23', 
           'WSP_mean_frcst_24',  'TMP_mean_frcst_1',  'TMP_mean_frcst_2',  'TMP_mean_frcst_3',
           'TMP_mean_frcst_4',  'TMP_mean_frcst_5',  'TMP_mean_frcst_6',  'TMP_mean_frcst_7',  
           'TMP_mean_frcst_8',  'TMP_mean_frcst_9',  'TMP_mean_frcst_10',  'TMP_mean_frcst_11', 
           'TMP_mean_frcst_12',  'TMP_mean_frcst_13',  'TMP_mean_frcst_14',  'TMP_mean_frcst_15',
           'TMP_mean_frcst_16',  'TMP_mean_frcst_17',  'TMP_mean_frcst_18',  'TMP_mean_frcst_19',  
           'TMP_mean_frcst_20',  'TMP_mean_frcst_21',  'TMP_mean_frcst_22',  'TMP_mean_frcst_23', 
           'TMP_mean_frcst_24']

Añadamos los datos de pronostico de las siguientes variables:

In [28]:
lista_frcst = ["RH_mean","WSP_mean","TMP_mean"]

In [29]:
for item in lista_frcst:
    for i in range(1, 25):
        col_name = str( item + "_frcst_" + str(i))
        data_hour_merge[col_name] = data_hour_merge[item].shift(i)
#data_hour_merge.dropna(inplace=True)

In [30]:
data_hour_merge = data_hour_merge.tail(-24)
data_hour_merge = data_hour_merge.fillna(data_hour_merge.mean())

In [31]:
data_hour_merge = data_hour_merge[O3_corr]

Guardemos:

In [32]:
data_hour_merge.to_csv(path_final,sep=',', encoding='utf-8',index=False)

In [40]:
data_hour_merge.columns.shape

(185,)